In [1]:
# #Python Libraries
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels
import pandas_profiling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import time
import requests
import datetime

import missingno as msno
import gc

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# #X_trainval, X_test, y_trainval, y_test = train_test_split(X, y)
# #X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from sklearn.cross_validation import train_test_split
import xgboost as xgb

c:\users\karti\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Ensemble - My XGBoost Model with Kaggle LB Train subset and Custome Features with https://www.kaggle.com/cttsai/blend-app-channel-and-app-mean/code

In [1]:
!wc -l model_3_xbg_submission_v4.csv

18790470 model_3_xbg_submission_v4.csv


In [2]:
!wc -l model_kaggle_lb_submission_appmean.csv

18790470 model_kaggle_lb_submission_appmean.csv


In [4]:
df_1 = pd.read_csv("model_3_xbg_submission_v4.csv")
df_2 = pd.read_csv("model_kaggle_lb_submission_appmean.csv")

In [14]:
df_1.head()

,click_id,is_attributed
0,0,0.000199
1,1,0.000117
2,2,0.000155
3,3,0.000135
4,4,0.000178


In [15]:
df_2.head()

,click_id,is_attributed
0,0,0.001152
1,1,0.001221
2,2,0.000111
3,3,0.000228
4,4,0.000106


In [16]:
sub = pd.DataFrame()
sub['click_id'] = df_1['click_id']

In [17]:
weight_df1 = 0.5
weight_df2 = 0.5
sub['is_attributed'] = (df_1['is_attributed']*weight_df1 + df_2['is_attributed']*weight_df2)

In [18]:
sub.head()

,click_id,is_attributed
0,0,0.000771
1,1,0.000779
2,2,0.000128
3,3,0.000191
4,4,0.000135


In [19]:
sub.to_csv('model_4_ensemble_xbg_kagglelbappmean_subv1.csv',index=False)

In [20]:
!head model_4_ensemble_xbg_kagglelbappmean_subv1.csv

click_id,is_attributed
0,0.0007708096739402926
1,0.0007790307139447393
2,0.0001282442026580456
3,0.00019059597977662886
4,0.00013496466976006133
5,0.00039491701885281883
6,0.0001990566928027148
7,0.0007400964701527287
9,0.0003498106948528188


In [21]:
!wc -l model_4_ensemble_xbg_kagglelbappmean_subv1.csv

18790470 model_4_ensemble_xbg_kagglelbappmean_subv1.csv


# Ensemble 2

In [1]:
!wc -l ensemble_data/*

  18790470 ensemble_data/ftrl_submission.csv
  18790470 ensemble_data/kartik_ensemble_1.csv
  18790470 ensemble_data/lgb_sub_tint.csv
  18790470 ensemble_data/sub_lgb_balanced99.csv
  18790470 ensemble_data/sub_mix.csv
  93952350 total


In [5]:
df_1 = pd.read_csv("ensemble_data/ftrl_submission.csv")
df_2 = pd.read_csv("ensemble_data/kartik_ensemble_1.csv")
df_3 = pd.read_csv("ensemble_data/lgb_sub_tint.csv")
df_4 = pd.read_csv("ensemble_data/sub_lgb_balanced99.csv")
df_5 = pd.read_csv("ensemble_data/sub_mix.csv")

In [6]:
sub = pd.DataFrame()
sub['click_id'] = df_2['click_id']

In [7]:
weight_df1 = 0.2
weight_df2 = 0.2
weight_df3 = 0.2
weight_df4 = 0.2
weight_df5 = 0.2
sub['is_attributed'] = (df_1['is_attributed']*weight_df1 + 
                        df_2['is_attributed']*weight_df2 + 
                        df_3['is_attributed']*weight_df3 +
                        df_4['is_attributed']*weight_df4 + 
                        df_5['is_attributed']*weight_df5)

In [8]:
sub.to_csv('model_5_ensemble_5csvs.csv',index=False)

In [9]:
!head model_5_ensemble_5csvs.csv

click_id,is_attributed
0,0.16122287281823855
1,0.16925022380114538
2,0.058899402505327596
3,0.12454995430063484
4,0.05288637108890904
5,0.17074734248496462
6,0.08237688101932572
7,0.19237944852967043
9,0.1427302100946004


# Ensemble 3

In [6]:
"""
xgb - https://www.kaggle.com/pranav84/xgboost-on-hist-mode-ip-addresses-dropped
ftrl - https://www.kaggle.com/ogrellier/ftrl-in-chunck
nn - https://www.kaggle.com/shujian/mlp-starter?scriptVersionId=2754301
lgb - https://www.kaggle.com/pranav84/lightgbm-fixing-unbalanced-data-val-auc-0-977?scriptVersionId=2761828
usam - https://www.kaggle.com/cartographic/undersampler
means - https://www.kaggle.com/prashantkikani/weighted-app-chanel-os

"""
# #LOGIT_WEIGHT = .5 #0.9640
LOGIT_WEIGHT = .8

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.special import expit, logit

almost_zero = 1e-10
almost_one = 1 - almost_zero

"""
Ensemble 3.1 - 0.9640
models = {
  'xgb  ':  "ensemble_data/sub_xgb_hist_pos_weight_9591.csv",
  'ftrl1':  "ensemble_data/ftrl_submission.csv",
  'nn   ':  "ensemble_data/sub_mlp_9502.csv",
  'lgb  ':  "ensemble_data/sub_lgb_balanced99_9631.csv",
  'usam ':  "ensemble_data/pred_9562.csv",
  'means':  "ensemble_data/subnew_9565.csv",
  'ftrl2':  "ensemble_data/ftrl_submission.csv"
  }
  
weights = {
  'xgb  ':  .08,
  'ftrl1':  .04,
  'nn   ':  .05,
  'lgb  ':  .65,
  'usam ':  .05,
  'means':  .07,
  'ftrl2':  .06
  }
"""

"""
Ensemble 3.2 - 0.9642
models = {
  'xgb  ':  "ensemble_data/xgb_sub_9610.csv",
  'ftrl1':  "ensemble_data/ftrl_submission_9606.csv",
  'nn   ':  "ensemble_data/sub_mlp_9502.csv",
  'lgb  ':  "ensemble_data/sub_lgb_balanced99_9631.csv",
  'usam ':  "ensemble_data/pred_9562.csv",
  'means':  "ensemble_data/subnew_9565.csv",
  'ftrl2':  "ensemble_data/ftrl_submission_9606.csv"
  }
  
weights = {
  'xgb  ':  .10,
  'ftrl1':  .04,
  'nn   ':  .05,
  'lgb  ':  .60,
  'usam ':  .05,
  'means':  .07,
  'ftrl2':  .09
  }
  
"""

"""
Ensemble 3.3 - Same as the one above without the Neural Network - 0.9651
models = {
  'xgb  ':  "ensemble_data/xgb_sub_9610.csv",
  'ftrl1':  "ensemble_data/ftrl_submission_9606.csv",
  'lgb  ':  "ensemble_data/sub_lgb_balanced99_9631.csv",
  'usam ':  "ensemble_data/pred_9562.csv",
  'means':  "ensemble_data/subnew_9565.csv",
  'ftrl2':  "ensemble_data/ftrl_submission_9606.csv"
  }
  
weights = {
  'xgb  ':  .15,
  'ftrl1':  .04,
  'lgb  ':  .60,
  'usam ':  .05,
  'means':  .07,
  'ftrl2':  .09
  }
"""

"""
Ensemble 3.4 - Same as the one above, modified weights - 0.9653
models = {
  'xgb  ':  "ensemble_data/xgb_sub_9610.csv",
  'ftrl1':  "ensemble_data/ftrl_submission_9606.csv",
  'lgb  ':  "ensemble_data/sub_lgb_balanced99_9631.csv",
  'means':  "ensemble_data/subnew_9565.csv",
  'ftrl2':  "ensemble_data/ftrl_submission_9606.csv"
  }
  
weights = {
  'xgb  ':  .20,
  'ftrl1':  .06,
  'lgb  ':  .60,
  'means':  .05,
  'ftrl2':  .09
  }
"""
"""
Ensemble 3.5 - Same as the one above, modified weights - 0.9680
models = {
  'xgb  ':  "ensemble_data/xgb_sub_9635.csv",
  'ftrl1':  "ensemble_data/ftrl_submission_9606.csv",
  'lgb  ':  "ensemble_data/sub_lgb_balanced99_9667.csv",
  'ftrl2':  "ensemble_data/ftrl_submission_9606.csv"
  }
  
weights = {
  'xgb  ':  .25,
  'ftrl1':  .06,
  'lgb  ':  .60,
  'ftrl2':  .09
  }
"""

""" - 0.9684
LOGIT_WEIGHT = .8
models = {

    'xgb':  "ensemble_data/xgb_sub_9645.csv",
    'ftrl1':  "ensemble_data/wordbatch_fm_ftrl_9615.csv",
    'lgb':  "ensemble_data/sub_lgb_balanced99_9675.csv",
    'dl_support': "ensemble_data/dl_support_9653.csv"
}
  
weights = {
    'xgb':  .10,
    'ftrl1':  .10,
    'lgb':  .60,
    'dl_support': .20
}
"""

LOGIT_WEIGHT = .2
models = {

    'xgb':  "ensemble_data/xgb_sub_9645.csv",
    'ftrl1':  "ensemble_data/wordbatch_fm_ftrl_9615.csv",
    'lgb':  "ensemble_data/sub_lgb_balanced99_9675.csv",
    'dl_support': "ensemble_data/dl_support_9653.csv"
}
  
weights = {
    'xgb':  .10,
    'ftrl1':  .10,
    'lgb':  .60,
    'dl_support': .20
}
print(sum(weights.values()))


subs = {m:pd.read_csv(models[m]) for m in models}
first_model = list(models.keys())[0]
n = subs[first_model].shape[0]

ranks = {s:subs[s]['is_attributed'].rank()/n for s in subs}
logits = {s:subs[s]['is_attributed'].clip(almost_zero,almost_one).apply(logit) for s in subs}

logit_avg = 0
rank_avg = 0
for m in models:
    s = logits[m].std()
    print(m, s)
    logit_avg = logit_avg + weights[m]*logits[m] / s
    rank_avg = rank_avg + weights[m]*ranks[m]

logit_rank_avg = logit_avg.rank()/n
final_avg = LOGIT_WEIGHT*logit_rank_avg + (1-LOGIT_WEIGHT)*rank_avg

final_sub = pd.DataFrame()
final_sub['click_id'] = subs[first_model]['click_id']
final_sub['is_attributed'] = final_avg

final_sub.to_csv("sub_kartik_mix_v6.csv", index=False)

1.0
xgb 1.6089018201524738
ftrl1 1.7997105120220467
lgb 1.6161302763780279
dl_support 2.1093812464319406


In [7]:
!wc -l sub_kartik_mix_v6.csv

18790470 sub_kartik_mix_v6.csv


In [21]:
weights = {
  'xgb  ':  .08,
  'ftrl1':  .04,
  'nn   ':  .05,
  'lgb  ':  .65,
  'usam ':  .05,
  'means':  .07,
  'ftrl2':  .06
  }
  
print(sum(weights.values()))

1.0000000000000002


In [5]:
weights = {
  'xgb  ':  .10,
  'ftrl1':  .04,
  'nn   ':  .05,
  'lgb  ':  .60,
  'usam ':  .05,
  'means':  .07,
  'ftrl2':  .09
  }
print(sum(weights.values()))

1.0000000000000002


In [10]:
weights = {
  'xgb  ':  .15,
  'ftrl1':  .04,
  'lgb  ':  .60,
  'usam ':  .05,
  'means':  .07,
  'ftrl2':  .09
  }
print(sum(weights.values()))

1.0000000000000002


In [15]:
weights = {
  'xgb  ':  .15,
  'ftrl1':  .06,
  'lgb  ':  .60,
  'means':  .10,
  'ftrl2':  .09
  }

print(sum(weights.values()))


0.9999999999999999
